## Examplifying the use of the IAE code
### This is a simple application of the IAE code on 1D Gaussian-shaped signals
### It features:
    - An example of the learning stage, showing how to choose the main hyperparameters
    - An illustration of the way the learnt model can be used to regularized a simple inverse problem (here denoising)
other examples to come soon

In [ ]:
## Required imports

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sys

PATH ='../codes'
sys.path.insert(1,PATH)

import IAE_JAX as mld
import DataGeneration as dg

np.random.seed(1)

In [ ]:
# Let us create the data

train_samp_number = 100
validation_samp_number = 100
test_samp_number = 100
t_length = 50

Output = dg.GaussianShaped(t_length=t_length, t_start=0, pmin=15, pmax=35, train_samp_number=train_samp_number,
                           validation_samp_number=validation_samp_number, test_samp_number=test_samp_number, 
                           law='linear')

AnchorPoints = Output["Psi"][[0,1],:] # Define the anchor points

plt.figure(figsize=(20,10))
plt.plot(AnchorPoints[0:2,:].T,'k--',lw=8,alpha=0.5,label='Anchor points')
plt.plot(Output["X_train"][0:10,:].T,lw = 4)
plt.legend()

In [ ]:
mu = 1 # Regularization parameter
step_size = 1e-3 # gradient step size
niter = 5000 # number of iterations or epochs

vNL = [2,3,4]

for NL in vNL:  # Learning with various numbers of layers

    if NL == 2:
        out0 = None # out0 is a first guess model
        
    fname = 'model_L'+np.str(NL) # filename of the saved model (pickle file)
    NSize = t_length*np.ones((NL+1,),dtype='int') # shape of the network
    LearnFunc = mld.IAE(fname=fname, Model=out0, simplex=True,
                                   AnchorPoints=AnchorPoints, NSize=NSize, step_size=step_size, reg_parameter=mu, 
                                   niter=niter, active_forward='lRelu', active_backward='lRelu', reg_inv=1e-8, 
                                   verb=True)   # Defines the learning class
    learnt_params, outval = LearnFunc.learning_stage(Output["X_train"], XValidation= Output["X_valid"], batch_size=100)  # learning stage
    out0 = mld.load_model(fname)
    plt.figure(figsize=[12,10])
    plt.loglog(outval["total_cost"], lw=8, alpha=0.25)
    plt.loglog(outval["samp_cost"], '.-', lw=4, alpha=0.25)
    plt.loglog(outval["trans_cost"],'--', lw=4, alpha=0.25)
    plt.legend(["total","sample-based","tf-based"])
    plt.title('Optim adam - NLayersns {:.2e}'.format(NL))

### Projection onto the barycentric span and robustness to noise

In [ ]:
fname='model_L4' # MODEL
ampp = 1. # AMPLITUDE
noise_level = 0.01 # NOISE LEVEL
Xb = ampp*Output["X_test"][0:1,:] + noise_level*np.random.randn(1,50) # ADD SOME NOISE

model = mld.load_model(fname)
LearnFunc = mld.IAE(Model = model) # niter is the number of iterations

rec_clean = LearnFunc.barycentric_span_projection(Xb,niter=1000)["XRec"].squeeze() # projection onto the barycentric span



In [ ]:
plt.figure(figsize=[12,10])
plt.plot(Xb.squeeze(),lw=10,alpha=0.25,label="noisy measurement")
plt.plot(ampp*Output["X_test"][0:1,:].squeeze(),'r--',lw=6,alpha=0.75,label='input signal')
plt.plot(rec_clean,'k',lw=3,alpha=0.8,label="clean projection")
plt.plot(ampp*Output["X_test"][0:1,:].squeeze()-rec_clean,'g--',lw=4,alpha=0.75,label="residual error")
plt.title('Reconstruction - '+fname)
plt.legend()
